<a href="https://colab.research.google.com/github/evalucre/evalucre/blob/main/AlgoTrade_Ensemble_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoofinancials

!pip install yahoo_fin
!pip install requests_html

!pip install yfinance 

import numpy as np
import time
import os 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import pandas as pd
import datetime as datetime
from yahoofinancials import YahooFinancials
import yahoo_fin.stock_info as si
import yfinance as yf

import requests
from math import floor
from termcolor import colored as cl

from google.colab import drive
drive.mount('/content/drive')

plt.rcParams['figure.figsize'] = (20, 10)
plt.style.use('fivethirtyeight')


In [ ]:
def get_ticker_list(grouped_stocks):
  if grouped_stocks == 'dow':
    ticker_list = si.tickers_dow()

  elif  grouped_stocks == 'sp500':
    ticker_list = si.tickers_sp500()

  elif grouped_stocks == 'nasdaq':
    ticker_list = si.tickers_nasdaq()

  elif grouped_stocks == 'tsx':
    ticker_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tsx_tickers.csv')
    ticker_list = ticker_list.iloc[:,0].tolist()

  elif grouped_stocks == 'exp':
    ticker_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/exp_tickers.csv')  
    ticker_list = ticker_list.iloc[:,0].tolist()  

  elif grouped_stocks == 'tsx60':
    ticker_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tsx60_tickers.csv')  
    ticker_list = ticker_list.iloc[:,0].tolist()  

  else:
    ticker_list = "Input group of stocks not recognised"

  return ticker_list




In [ ]:
def get_trading_data(ticker_list, start_date, end_date, index_as_date, interval):
  
  trading_datas = {}
  
  for ticker in ticker_list:
    try:
      trading_datas[ticker] = si.get_data(ticker, start_date, end_date, index_as_date, interval)  
    except:
      pass

  trading_datas = pd.concat(trading_datas)

  trading_datas = trading_datas.reset_index(level=[0,1])\
                               .drop(labels='level_0', axis=1)\
                               .rename(columns={'level_1':'date'})\
                               .set_index('date')

  return trading_datas  


In [ ]:
def get_financial_data(ticker_list):
  
  cashflow_datas = {}
  balncsheet_datas = {}
  incomstment_datas = {}
  
  for ticker in ticker_list:
    cashflow_datas[ticker] = si.get_cash_flow(ticker) 
    balncsheet_datas[ticker] = si.get_balance_sheet(ticker)
    incomstment_datas[ticker] = si.get_income_statement(ticker) 


  cashflow_datas = pd.concat(cashflow_datas) 
  balncsheet_datas = pd.concat(balncsheet_datas)
  incomstment_datas = pd.concat(incomstment_datas)

  cashflow_datas = cashflow_datas.transpose()
  balncsheet_datas = balncsheet_datas.transpose()
  incomstment_datas = incomstment_datas.transpose()

  return cashflow_datas, balncsheet_datas, incomstment_datas 



In [ ]:
def get_selection(trading_datas):
  """
  Select best stocks (numb = 10) with right price volatility and variability for trading 
  or investing. When the preference is to identify stocks for investing, input 
  data should cover a longer time period (years). When trading is the focus, the
  input data should cover shorter time periods (like week, month, or quarters) 

  """

  trading_datas = trading_datas.loc[(trading_datas.volume >= 50000)]

  metric_Pstdv = {}
  metric_Rstdv = {}
  metric_Vmean = {}
  metric_Msize = {}

  trading_datas['prange'] = trading_datas['high'] - trading_datas['low']

  stocks = trading_datas.ticker.unique()
  for stock in stocks:
    dt = trading_datas.loc[(trading_datas.ticker == stock)]
    metric_Pstdv[stock] = dt.close.std()  #price standard deviation (high for investment stocks) - profitability indicator
    metric_Rstdv[stock] = dt.prange.std()  #prange standard deviation (high for trading stocks) - volatility indicator
    metric_Vmean[stock] = dt.volume.mean()  #mean of volume - liquidity indicator
    metric_Msize[stock] = (dt.volume.mean())*(dt.close.mean())  #measure of market size/capitalization

#Rank stocks using calculated metrics
  metric_Pstdv = pd.DataFrame(metric_Pstdv, index=[0])
  metric_Rstdv = pd.DataFrame(metric_Rstdv, index=[0])
  metric_Vmean = pd.DataFrame(metric_Vmean, index=[0])
  metric_Msize = pd.DataFrame(metric_Msize, index=[0])

  metric_Pstdv = metric_Pstdv.T.reset_index()\
                               .rename(columns={'index':'ticker'})\
                               .rename(columns={0:'Pstdv'})\
                               .sort_values(by=['Pstdv'], ascending = False)
                            
  
  metric_Rstdv = metric_Rstdv.T.reset_index()\
                               .rename(columns={'index':'ticker'})\
                               .rename(columns={0:'Rstdv'})\
                               .sort_values(by=['Rstdv'], ascending = False)
 

  metric_Vmean = metric_Vmean.T.reset_index()\
                               .rename(columns={'index':'ticker'})\
                               .rename(columns={0:'Vmean'})\
                               .sort_values(by=['Vmean'], ascending = False)


  metric_Msize = metric_Msize.T.reset_index()\
                               .rename(columns={'index':'ticker'})\
                               .rename(columns={0:'Msize'})\
                               .sort_values(by=['Msize'], ascending = False)
                          
#Select top 10 stocks with highest values of the metrics
  numb = 10
  metric_Pstdv = metric_Pstdv[:numb]['ticker'].reset_index(drop=True)      
  metric_Rstdv = metric_Rstdv[:numb]['ticker'].reset_index(drop=True)
  metric_Vmean = metric_Vmean[:numb]['ticker'].reset_index(drop=True)
  metric_Msize = metric_Msize[:numb]['ticker'].reset_index(drop=True) 


  tradin_select = metric_Rstdv[(pd.Series(metric_Rstdv.to_numpy()).isin(metric_Vmean.to_numpy()))]
  invest_select = metric_Pstdv[(pd.Series(metric_Pstdv.to_numpy()).isin(metric_Vmean.to_numpy()))]

  if len(tradin_select) == 0:
    tradin_select = metric_Rstdv

  if len(invest_select) == 0:
    invest_select = metric_Pstdv
  
  #Definitions:
  #tradin_select - combines volatility and high trade volumes
  #invest_select - long term growth and high capitalization
  #volatl_select - high volatility but does not necessarily have trade volumes

  return tradin_select, invest_select


In [ ]:
def get_macd(trading_datas, select_ticker, fastm, slowm, smooth):

  df = trading_datas[trading_datas['ticker'] == select_ticker]

  exp1 = df.close.ewm(span=fastm, adjust=False).mean()
  exp2 = df.close.ewm(span=slowm, adjust=False).mean()
  macd = pd.DataFrame(exp1 - exp2).rename(columns = {'close':'macd'})
  signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
  hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
  
  df = pd.concat([df, macd, signal, hist], join = 'inner', axis = 1)

  return df

In [ ]:
 def get_stochst(trading_datas, select_ticker, roll_low, roll_high, fasts, slows):
    """
    Fast stochastic calculation
    %K = (Current Close - Lowest Low)/
    (Highest High - Lowest Low) * 100
    %D = SMA of %K

    Slow stochastic calculation
    %K = %D of fast stochastic
    %D = SMA of %K

    When %K crosses above %D, buy signal 
    When the %K crosses below %D, sell signal
    """
# It seems like d_fast would be best for high volatility trading and d_slow for lower volatility investing

    df = trading_datas[trading_datas['ticker'] == select_ticker]

# Set minimum low and maximum high of the k stoch
    low_min  = df.low.rolling( window = roll_low ).min()
    high_max = df.high.rolling( window = roll_high ).max()

# Fast Stochastic
    df['k_fast'] = 100 * (df.close - low_min)/(high_max - low_min)
    df['d_fast'] = df['k_fast'].rolling(window = fasts).mean()

# Slow Stochastic
    df['k_slow'] = df['d_fast']
    df['d_slow'] = df['k_slow'].rolling(window = slows).mean()

    return df


In [ ]:
def strategy_macd(macd_data):    
    buy_price = []
    sell_price = []
    macd_signal = []
    position = np.zeros(len(macd_data))
    signal = 0

    for i in range(len(macd_data)):
        if macd_data['macd'][i] > macd_data['signal'][i]:
            if signal != 1:
                buy_price.append(macd_data['close'][i])
                sell_price.append(np.nan)
                signal = 1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        elif macd_data['macd'][i] < macd_data['signal'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(macd_data['close'][i])
                signal = -1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            macd_signal.append(0)

    for i in range(len(macd_data)):
      if macd_signal[i] == 1:
        position[i] = 1
      elif macd_signal[i] == -1:
        position[i] = 0
      elif (macd_signal[i] == 0) and (i == 0):
        position[i] = 0
      else:
        position[i] = position[i-1]

    macd_signal = pd.DataFrame(macd_signal).rename(columns = {0:'macd_signal'}).set_index(macd_data.index)
    position = pd.DataFrame(position).rename(columns = {0:'macd_position'}).set_index(macd_data.index)
    buy_price = pd.DataFrame(buy_price).rename(columns = {0:'buy_price'}).set_index(macd_data.index)
    sell_price = pd.DataFrame(sell_price).rename(columns = {0:'sell_price'}).set_index(macd_data.index)

    macd_decisions = pd.concat([macd_data['close'], macd_data['macd'], macd_data['signal'], macd_signal, position, buy_price, sell_price], join = 'inner', axis = 1)


    return macd_decisions

In [ ]:
def strategy_stochst(stochst_data):    
    buy_price1, buy_price2, buy_price3 = [], [], []
    sell_price1, sell_price2, sell_price3 = [], [], []
    stochst_signal1, stochst_signal2, stochst_signal3 = [], [], []
    position1 = np.zeros(len(stochst_data))
    position2 = np.zeros(len(stochst_data))
    position3 = np.zeros(len(stochst_data))
    
    signal = 0
    for i in range(len(stochst_data)):
        if (stochst_data['k_fast'][i] < 40) and (stochst_data['d_fast'][i] > stochst_data['d_slow'][i]):
            if signal != 1:
                buy_price1.append(stochst_data['close'][i])
                sell_price1.append(np.nan)
                signal = 1
                stochst_signal1.append(signal)
            else:
                buy_price1.append(np.nan)
                sell_price1.append(np.nan)
                stochst_signal1.append(0)
        elif (stochst_data['k_fast'][i] > 80) and (stochst_data['d_fast'][i] < stochst_data['d_slow'][i]):
            if signal != -1:
                buy_price1.append(np.nan)
                sell_price1.append(stochst_data['close'][i])
                signal = -1
                stochst_signal1.append(signal)
            else:
                buy_price1.append(np.nan)
                sell_price1.append(np.nan)
                stochst_signal1.append(0)
        else:
            buy_price1.append(np.nan)
            sell_price1.append(np.nan)
            stochst_signal1.append(0)

    signal = 0
    for i in range(len(stochst_data)):
        if (stochst_data['k_fast'][i] < 30) and (stochst_data['d_fast'][i] > stochst_data['d_slow'][i]):
            if signal != 1:
                buy_price2.append(stochst_data['close'][i])
                sell_price2.append(np.nan)
                signal = 1
                stochst_signal2.append(signal)
            else:
                buy_price2.append(np.nan)
                sell_price2.append(np.nan)
                stochst_signal2.append(0)
        elif (stochst_data['k_fast'][i] > 65) and (stochst_data['d_fast'][i] < stochst_data['d_slow'][i]):
            if signal != -1:
                buy_price2.append(np.nan)
                sell_price2.append(stochst_data['close'][i])
                signal = -1
                stochst_signal2.append(signal)
            else:
                buy_price2.append(np.nan)
                sell_price2.append(np.nan)
                stochst_signal2.append(0)
        else:
            buy_price2.append(np.nan)
            sell_price2.append(np.nan)
            stochst_signal2.append(0)

    signal = 0
    for i in range(len(stochst_data)):
        if (stochst_data['k_fast'][i] < 25) and (stochst_data['d_fast'][i] > stochst_data['d_slow'][i]):
            if signal != 1:
                buy_price3.append(stochst_data['close'][i])
                sell_price3.append(np.nan)
                signal = 1
                stochst_signal3.append(signal)
            else:
                buy_price3.append(np.nan)
                sell_price3.append(np.nan)
                stochst_signal3.append(0)
        elif (stochst_data['k_fast'][i] > 75) and (stochst_data['d_fast'][i] < stochst_data['d_slow'][i]):
            if signal != -1:
                buy_price3.append(np.nan)
                sell_price3.append(stochst_data['close'][i])
                signal = -1
                stochst_signal3.append(signal)
            else:
                buy_price3.append(np.nan)
                sell_price3.append(np.nan)
                stochst_signal3.append(0)
        else:
            buy_price3.append(np.nan)
            sell_price3.append(np.nan)
            stochst_signal3.append(0)



    for i in range(len(stochst_data['close'])):
      if stochst_signal1[i] == 1:
        position1[i] = 1
      elif stochst_signal1[i] == -1:
        position1[i] = 0
      elif (stochst_signal1[i] == 0) and (i == 0):
        position1[i] = 0
      else:
        position1[i] = position1[i-1]


    for i in range(len(stochst_data['close'])):
      if stochst_signal2[i] == 1:
        position2[i] = 1
      elif stochst_signal2[i] == -1:
        position2[i] = 0
      elif (stochst_signal2[i] == 0) and (i == 0):
        position2[i] = 0
      else:
        position2[i] = position2[i-1]


    for i in range(len(stochst_data['close'])):
      if stochst_signal3[i] == 1:
        position3[i] = 1
      elif stochst_signal3[i] == -1:
        position3[i] = 0
      elif (stochst_signal3[i] == 0) and (i == 0):
        position3[i] = 0
      else:
        position3[i] = position3[i-1]


    stochst_signal1 = pd.DataFrame(stochst_signal1).rename(columns = {0:'stochst_signal1'}).set_index(stochst_data.index)
    position1 = pd.DataFrame(position1).rename(columns = {0:'stochst_position1'}).set_index(stochst_data.index)
    buy_price1 = pd.DataFrame(buy_price1).rename(columns = {0:'buy_price1'}).set_index(stochst_data.index)
    sell_price1 = pd.DataFrame(sell_price1).rename(columns = {0:'sell_price1'}).set_index(stochst_data.index)

    stochst_signal2 = pd.DataFrame(stochst_signal2).rename(columns = {0:'stochst_signal2'}).set_index(stochst_data.index)
    position2 = pd.DataFrame(position2).rename(columns = {0:'stochst_position2'}).set_index(stochst_data.index)
    buy_price2 = pd.DataFrame(buy_price2).rename(columns = {0:'buy_price2'}).set_index(stochst_data.index)
    sell_price2 = pd.DataFrame(sell_price2).rename(columns = {0:'sell_price2'}).set_index(stochst_data.index)

    stochst_signal3 = pd.DataFrame(stochst_signal3).rename(columns = {0:'stochst_signal3'}).set_index(stochst_data.index)
    position3 = pd.DataFrame(position3).rename(columns = {0:'stochst_position3'}).set_index(stochst_data.index)
    buy_price3 = pd.DataFrame(buy_price3).rename(columns = {0:'buy_price3'}).set_index(stochst_data.index)
    sell_price3 = pd.DataFrame(sell_price3).rename(columns = {0:'sell_price3'}).set_index(stochst_data.index)


    stochst_decisions = pd.concat([stochst_data['close'], stochst_signal1, position1, stochst_signal2, position2, stochst_signal3, position3], join = 'inner', axis = 1)


    return stochst_decisions

In [ ]:
def scenarios_invest(macd_decisions, stochst_decisions):
  """
  Investment Scenarios
  -High Risk (HR): based on only smap and smav 
  -Medium Risk (MR): based on smap, smav, macd, and stochastic
  -Low Risk (LR): based on smap, smav, macd, stochastic, and bollinger
  """

  df = pd.concat([macd_decisions, stochst_decisions], join = 'inner', axis = 1)

  HR_signal = []
  MR_signal = []
  LR_signal = []
  HR_position = np.zeros(len(df))
  MR_position = np.zeros(len(df))
  LR_position = np.zeros(len(df))


#HR SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if (df['macd_signal'][i] == 1) and (df['stochst_signal1'][i] == 1):
          if signal != 1:
              signal = 1
              HR_signal.append(signal)
          else:
              HR_signal.append(0)
      elif (df['macd_signal'][i] == -1) and (df['stochst_signal1'][i] == -1):
          if signal != -1:
              signal = -1
              HR_signal.append(signal)
          else:
              HR_signal.append(0)
      else:
          HR_signal.append(0)

#MR SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if (df['macd_signal'][i] == 1) and (df['stochst_signal2'][i] == 1):
          if signal != 1:
              signal = 1
              MR_signal.append(signal)
          else:
              MR_signal.append(0)
      elif (df['macd_signal'][i] == -1) and (df['stochst_signal2'][i] == -1):
          if signal != -1:
              signal = -1
              MR_signal.append(signal)
          else:
              MR_signal.append(0)
      else:
          MR_signal.append(0)

#LR SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if (df['macd_signal'][i] == 1) and (df['stochst_signal3'][i] == 1):
          if signal != 1:
              signal = 1
              LR_signal.append(signal)
          else:
              LR_signal.append(0)
      elif (df['macd_signal'][i] == -1) and (df['stochst_signal3'][i] == -1):
          if signal != -1:
              signal = -1
              LR_signal.append(signal)
          else:
              LR_signal.append(0)
      else:
          LR_signal.append(0)


#HR POSITION CONDITIONS
  for i in range(len(df)):
    if HR_signal[i] == 1:
      HR_position[i] = 1
    elif HR_signal[i] == -1:
      HR_position[i] = 0
    elif (HR_signal[i] == 0) and (i == 0):
      HR_position[i] = 0
    else:
      HR_position[i] = HR_position[i-1]

#MR POSITION CONDITIONS
  for i in range(len(df)):
    if MR_signal[i] == 1:
      MR_position[i] = 1
    elif MR_signal[i] == -1:
      MR_position[i] = 0
    elif (MR_signal[i] == 0) and (i == 0):
      MR_position[i] = 0
    else:
      MR_position[i] = MR_position[i-1]

#LR POSITION CONDITIONS
  for i in range(len(df)):
    if LR_signal[i] == 1:
      LR_position[i] = 1
    elif LR_signal[i] == -1:
      LR_position[i] = 0
    elif (LR_signal[i] == 0) and (i == 0):
      LR_position[i] = 0
    else:
      LR_position[i] = LR_position[i-1]


  HR_signal = pd.DataFrame(HR_signal).rename(columns = {0:'HR_signal'}).set_index(df.index)
  MR_signal = pd.DataFrame(MR_signal).rename(columns = {0:'MR_signal'}).set_index(df.index)
  LR_signal = pd.DataFrame(LR_signal).rename(columns = {0:'LR_signal'}).set_index(df.index)

  HR_position = pd.DataFrame(HR_position).rename(columns = {0:'HR_position'}).set_index(df.index)
  MR_position = pd.DataFrame(MR_position).rename(columns = {0:'MR_position'}).set_index(df.index)
  LR_position = pd.DataFrame(LR_position).rename(columns = {0:'LR_position'}).set_index(df.index)

# invest_table = pd.concat([df['close'], HR_signal, MR_signal, LR_signal, HR_position, MR_position, LR_position], join = 'inner', axis = 1)
  invest_table = pd.concat([macd_decisions['close'], HR_signal, MR_signal, LR_signal, HR_position, MR_position, LR_position], join = 'inner', axis = 1)

  return invest_table

In [ ]:
def scenarios_tradin(macd_decisions, stochst_decisions):

  """
  Trading Scenarios (all considered high risk)
  -Short View (SV): uses one and two weeks trends (5 and 10 days) ...smap, smav, boband
  -Medium View (MV): uses one and two weeks trends (5 and 10 days)...smap, smav, macd, stochst
  -Long View (LV): uses current price and ten weeks trends (close and 50 days)...smap, smav, macd, boband
  """


  df = pd.concat([macd_decisions, stochst_decisions], join = 'inner', axis = 1)

  SV_signal = []
  MV_signal = []
  LV_signal = []
  SV_position = np.zeros(len(df))
  MV_position = np.zeros(len(df))
  LV_position = np.zeros(len(df))

  #SV SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if (df['macd_signal'][i] == 1) and (df['stochst_signal1'][i] == 1):
          if signal != 1:
              signal = 1
              SV_signal.append(signal)
          else:
              SV_signal.append(0)
      elif (df['macd_signal'][i] == -1) and (df['stochst_signal1'][i] == -1):
          if signal != -1:
              signal = -1
              SV_signal.append(signal)
          else:
              SV_signal.append(0)
      else:
          SV_signal.append(0)

  #MV SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if (df['macd_signal'][i] == 1) and (df['stochst_signal2'][i] == 1):
          if signal != 1:
              signal = 1
              MV_signal.append(signal)
          else:
              MV_signal.append(0)
      elif (df['macd_signal'][i] == -1) and (df['stochst_signal2'][i] == -1):
          if signal != -1:
              signal = -1
              MV_signal.append(signal)
          else:
              MV_signal.append(0)
      else:
          MV_signal.append(0)

  #LV SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if (df['macd_signal'][i] == 1) and (df['stochst_signal3'][i] == 1):
          if signal != 1:
              signal = 1
              LV_signal.append(signal)
          else:
              LV_signal.append(0)
      elif (df['macd_signal'][i] == -1) and (df['stochst_signal3'][i] == -1):
          if signal != -1:
              signal = -1
              LV_signal.append(signal)
          else:
              LV_signal.append(0)
      else:
          LV_signal.append(0)


#SV POSITION CONDITIONS
  for i in range(len(df)):
    if SV_signal[i] == 1:
      SV_position[i] = 1
    elif SV_signal[i] == -1:
      SV_position[i] = 0
    elif (SV_signal[i] == 0) and (i == 0):
      SV_position[i] = 0
    else:
      SV_position[i] = SV_position[i-1]

#MV POSITION CONDITIONS
  for i in range(len(df)):
    if MV_signal[i] == 1:
      MV_position[i] = 1
    elif MV_signal[i] == -1:
      MV_position[i] = 0
    elif (MV_signal[i] == 0) and (i == 0):
      MV_position[i] = 0
    else:
      MV_position[i] = MV_position[i-1]

#LV POSITION CONDITIONS
  for i in range(len(df)):
    if LV_signal[i] == 1:
      LV_position[i] = 1
    elif LV_signal[i] == -1:
      LV_position[i] = 0
    elif (LV_signal[i] == 0) and (i == 0):
      LV_position[i] = 0
    else:
      LV_position[i] = LV_position[i-1]




  SV_signal = pd.DataFrame(SV_signal).rename(columns = {0:'SV_signal'}).set_index(df.index)
  MV_signal = pd.DataFrame(MV_signal).rename(columns = {0:'MV_signal'}).set_index(df.index)
  LV_signal = pd.DataFrame(LV_signal).rename(columns = {0:'LV_signal'}).set_index(df.index)

  SV_position = pd.DataFrame(SV_position).rename(columns = {0:'SV_position'}).set_index(df.index)
  MV_position = pd.DataFrame(MV_position).rename(columns = {0:'MV_position'}).set_index(df.index)
  LV_position = pd.DataFrame(LV_position).rename(columns = {0:'LV_position'}).set_index(df.index)

  #tradin_table = pd.concat([df['close'], SV_signal, MV_signal, LV_signal, SV_position, MV_position, LV_position], join = 'inner', axis = 1)  
  tradin_table = pd.concat([macd_decisions['close'], SV_signal, MV_signal, LV_signal, SV_position, MV_position, LV_position], join = 'inner', axis = 1)

  return tradin_table


In [ ]:
"""
WORKFLOW EXECUTION SCRIPT: THIS IS USED TO RUN THE TRADING MODULES ON OFFLINE BASIS

"""
#Selection of stocks for trading or investment
grouped_stocks='tsx'
ticker_list = get_ticker_list(grouped_stocks)                                             #Other currently available groups are: 'tsx', 'sp500', 'nasdaq' 

#Get (historical) data on longterm performance os stock - 20 years preferable
start_date = (datetime.date.today() + datetime.timedelta(days=-4)).strftime("%m/%d/%Y")                                                            #Input format is: month/day/Year
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime("%m/%d/%Y")
trading_datas = get_trading_data(ticker_list, start_date, end_date, index_as_date = True, interval = "1d")

ticker_list = trading_datas['ticker'].unique()

#Select stocks based on attributes/performance for investment or trading
tradin_select, _ = get_selection(trading_datas)

output_text = str(grouped_stocks) +  "_tradin_watchlist.txt"
with open('/content/drive/MyDrive/Colab Notebooks/{}'.format(str(output_text)), "w") as text_file:
  print(tradin_select, file=text_file)

ticker_list = tradin_select
start_date = (datetime.date.today() + datetime.timedelta(days=-93)).strftime("%m/%d/%Y")                                                            #Input format is: month/day/Year
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime("%m/%d/%Y")
trading_datas = get_trading_data(ticker_list, start_date, end_date, index_as_date = True, interval = "1d")



#############################################TRADING#####################################################################################################################
#Call algotrading methods for selected stocks for Day-Trading
fastm, slowm, smooth = 6, 17, 10                    #MACD parameters
roll_low, roll_high, fasts, slows = 5, 14, 7, 14    #Stochastic bands parameters

macd_datat, stochst_datat = {}, {}

for select_ticker in tradin_select:
  macd_datat[select_ticker] = get_macd(trading_datas, select_ticker, fastm, slowm, smooth)
  stochst_datat[select_ticker] = get_stochst(trading_datas, select_ticker, roll_low, roll_high, fasts, slows)

macd_datat = pd.concat(macd_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
stochst_datat = pd.concat(stochst_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')

#Evaluate the strategy of each trading decision method
macd_decisiont, stochst_decisiont = {}, {}
for select_ticker in tradin_select:
  macd_decisiont[select_ticker] = strategy_macd(macd_data = macd_datat[macd_datat['ticker'] == select_ticker])
  stochst_decisiont[select_ticker] = strategy_stochst(stochst_data = stochst_datat[stochst_datat['ticker'] == select_ticker])

macd_decisiont = pd.concat(macd_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
stochst_decisiont = pd.concat(stochst_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

#Evaluate trading scenarios
tradin_table = {}
for select_ticker in tradin_select:
  tradin_table[select_ticker] = scenarios_tradin(macd_decisiont[macd_decisiont['ticker'] == select_ticker], stochst_decisiont[stochst_decisiont['ticker'] == select_ticker])

tradin_table = pd.concat(tradin_table).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

########################################################################################################################################################################


########################################################BACK TESTING####################################################################################################
#Trading
SV_buys = abs(sum(tradin_table['close'][(tradin_table['SV_signal']>0)]*tradin_table['SV_signal'][(tradin_table['SV_signal']>0)]))
SV_sales = abs(sum(tradin_table['close'][(tradin_table['SV_signal']<0)]*tradin_table['SV_signal'][(tradin_table['SV_signal']<0)]))            
MV_buys = abs(sum(tradin_table['close'][(tradin_table['MV_signal']>0)]*tradin_table['MV_signal'][(tradin_table['MV_signal']>0)]))
MV_sales = abs(sum(tradin_table['close'][(tradin_table['MV_signal']<0)]*tradin_table['MV_signal'][(tradin_table['MV_signal']<0)]))
LV_buys = abs(sum(tradin_table['close'][(tradin_table['LV_signal']>0)]*tradin_table['LV_signal'][(tradin_table['LV_signal']>0)]))
LV_sales = abs(sum(tradin_table['close'][(tradin_table['LV_signal']<0)]*tradin_table['LV_signal'][(tradin_table['LV_signal']<0)]))

SV_bSignals = abs(sum(tradin_table['SV_signal'][(tradin_table['SV_signal']>0)]))
SV_sSignals = abs(sum(tradin_table['SV_signal'][(tradin_table['SV_signal']<0)]))
MV_bSignals = abs(sum(tradin_table['MV_signal'][(tradin_table['MV_signal']>0)]))
MV_sSignals = abs(sum(tradin_table['MV_signal'][(tradin_table['MV_signal']<0)]))
LV_bSignals = abs(sum(tradin_table['LV_signal'][(tradin_table['LV_signal']>0)]))
LV_sSignals = abs(sum(tradin_table['LV_signal'][(tradin_table['LV_signal']<0)]))

try:
  SV_bPrice = SV_buys/SV_bSignals
  SV_sPrice = SV_sales/SV_sSignals
  SV_return = (SV_sPrice - SV_bPrice)*100/SV_bPrice
except:
  SV_return = 'nan' 

try:
  MV_bPrice = MV_buys/MV_bSignals
  MV_sPrice = MV_sales/MV_sSignals
  MV_return = (MV_sPrice - MV_bPrice)*100/MV_bPrice
except:
  MV_return = 'nan'

try:
  LV_bPrice = LV_buys/LV_bSignals
  LV_sPrice = LV_sales/LV_sSignals
  LV_return = (LV_sPrice - LV_bPrice)*100/LV_bPrice
except:
  LV_return = 'nan'



scenarios_backtest = pd.DataFrame({'buySignals': [SV_bSignals, MV_bSignals, LV_bSignals],
                        'sellSignals': [SV_sSignals, MV_sSignals, LV_sSignals], 
                        'Returns': [SV_return, MV_return, LV_return]}, index = ['SV', 'MV', 'LV'])
output_file = str(grouped_stocks) +  "_tradin_backtest"
scenarios_backtest.to_csv('/content/drive/MyDrive/Colab Notebooks/{}.csv'.format(output_file), header=True, index=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [ ]:
"""
WORKFLOW EXECUTION SCRIPT: THIS IS USED TO RUN THE INVESTING MODULES ON OFFLINE BASIS

"""
#Selection of stocks for trading or investment
grouped_stocks='tsx'
ticker_list = get_ticker_list(grouped_stocks)                                             #Other currently available groups are: 'tsx', 'sp500', 'nasdaq' 

#Get (historical) data on longterm performance os stock - 20 years preferable
start_date = (datetime.date.today() + datetime.timedelta(days=-93)).strftime("%m/%d/%Y")                                                            #Input format is: month/day/Year
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime("%m/%d/%Y")
trading_datas = get_trading_data(ticker_list, start_date, end_date, index_as_date = True, interval = "1d")

ticker_list = trading_datas['ticker'].unique()

#Select stocks based on attributes/performance for investment or trading
_, invest_select = get_selection(trading_datas)

output_text = str(grouped_stocks) +  "_invest_watchlist.txt"
with open('/content/drive/MyDrive/Colab Notebooks/{}'.format(str(output_text)), "w") as text_file:
  print(invest_select, file=text_file)

ticker_list = invest_select
start_date = (datetime.date.today() + datetime.timedelta(days=-366)).strftime("%m/%d/%Y")                                                            #Input format is: month/day/Year
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime("%m/%d/%Y")
trading_datas = get_trading_data(ticker_list, start_date, end_date, index_as_date = True, interval = "1d")



#############################################TRADING#####################################################################################################################
#Call algotrading methods for selected stocks for Day-Trading
fastm, slowm, smooth = 17, 35, 10                    #MACD parameters
roll_low, roll_high, fasts, slows = 14, 34, 16, 34    #Stochastic bands parameters

macd_datat, stochst_datat = {}, {}

for select_ticker in invest_select:
  macd_datat[select_ticker] = get_macd(trading_datas, select_ticker, fastm, slowm, smooth)
  stochst_datat[select_ticker] = get_stochst(trading_datas, select_ticker, roll_low, roll_high, fasts, slows)

macd_datat = pd.concat(macd_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
stochst_datat = pd.concat(stochst_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')

#Evaluate the strategy of each trading decision method
macd_decisiont, stochst_decisiont = {}, {}
for select_ticker in invest_select:
  macd_decisiont[select_ticker] = strategy_macd(macd_data = macd_datat[macd_datat['ticker'] == select_ticker])
  stochst_decisiont[select_ticker] = strategy_stochst(stochst_data = stochst_datat[stochst_datat['ticker'] == select_ticker])

macd_decisiont = pd.concat(macd_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
stochst_decisiont = pd.concat(stochst_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

#Evaluate trading scenarios
invest_table = {}
for select_ticker in invest_select:
  invest_table[select_ticker] = scenarios_invest(macd_decisiont[macd_decisiont['ticker'] == select_ticker], stochst_decisiont[stochst_decisiont['ticker'] == select_ticker])

invest_table = pd.concat(invest_table).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

########################################################################################################################################################################


########################################################BACK TESTING####################################################################################################
#Investing
HR_buys = abs(sum(tradin_table['close'][(tradin_table['HR_signal']>0)]*tradin_table['HR_signal'][(tradin_table['HR_signal']>0)]))
HR_sales = abs(sum(tradin_table['close'][(tradin_table['HR_signal']<0)]*tradin_table['HR_signal'][(tradin_table['HR_signal']<0)]))          
MR_buys = abs(sum(tradin_table['close'][(tradin_table['MR_signal']>0)]*tradin_table['MR_signal'][(tradin_table['MR_signal']>0)]))
MR_sales = abs(sum(tradin_table['close'][(tradin_table['MR_signal']<0)]*tradin_table['MR_signal'][(tradin_table['MR_signal']<0)]))
LR_buys = abs(sum(tradin_table['close'][(tradin_table['LR_signal']>0)]*tradin_table['LR_signal'][(tradin_table['LR_signal']>0)]))
LR_sales = abs(sum(tradin_table['close'][(tradin_table['LR_signal']<0)]*tradin_table['LR_signal'][(tradin_table['LR_signal']<0)]))


HR_bSignals = abs(sum(tradin_table['HR_signal'][(tradin_table['HR_signal']>0)]))
HR_sSignals = abs(sum(tradin_table['HR_signal'][(tradin_table['HR_signal']<0)]))
MR_bSignals = abs(sum(tradin_table['MR_signal'][(tradin_table['MR_signal']>0)]))
MR_sSignals = abs(sum(tradin_table['MR_signal'][(tradin_table['MR_signal']<0)]))
LR_bSignals = abs(sum(tradin_table['LR_signal'][(tradin_table['LR_signal']>0)]))
LR_sSignals = abs(sum(tradin_table['LR_signal'][(tradin_table['LR_signal']<0)]))

try:
  HR_bPrice = HR_buys/HR_bSignals
  HR_sPrice = HR_sales/HR_sSignals
  HR_return = (HR_sPrice - HR_bPrice)*100/HR_bPrice
except:
  HR_return = 'nan' 

try:
  MR_bPrice = MR_buys/MR_bSignals
  MR_sPrice = MR_sales/MR_sSignals
  MR_return = (MR_sPrice - MR_bPrice)*100/MR_bPrice
except:
  MR_return = 'nan'

try:
  LR_bPrice = LR_buys/LR_bSignals
  LR_sPrice = LR_sales/LR_sSignals
  LR_return = (LR_sPrice - LR_bPrice)*100/LR_bPrice
except:
  LR_return = 'nan'


scenarios_backtest = pd.DataFrame({'buySignals': [HR_bSignals, MR_bSignals, LR_bSignals],
                        'sellSignals': [HR_sSignals, MR_sSignals, LR_sSignals], 
                        'Returns': [HR_return, MR_return, LR_return]}, index = ['HR', 'MR', 'LR'])
output_file = str(grouped_stocks) +  "_invest_backtest"
scenarios_backtest.to_csv('/content/drive/MyDrive/Colab Notebooks/{}.csv'.format(output_file), header=True, index=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

KeyError: ignored